Import libs

In [1]:
from neo4j import GraphDatabase
import pathlib

Create connection

In [15]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
                
        return response
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="vovse")

Delete everything in graph

In [16]:
query = '''MATCH (n) DETACH DELETE n'''
out = conn.query(query)

Load computer data

In [17]:
query = '''
CALL apoc.load.json('%s/../jsondata/0001-computers.json') YIELD value
UNWIND value.computers AS c
MERGE (computer:Computer {sid:c.SID})
SET computer += c.Properties

// Local users
WITH c, computer
UNWIND c.LocalUsers AS lu
MERGE (localUser:LocalUser {name:lu.name, sid:lu.SID, computerName:computer.name})
MERGE (computer)-[:hasUser]->(localUser)

// Local groups
WITH c, computer
UNWIND c.LocalGroups AS lg
MERGE (localGroup:LocalGroup {name:lg.name, sid:lg.SID, computerName:computer.name})
MERGE (computer)-[:hasGroup]->(localGroup)

// Local group members
WITH lg, localGroup, computer
UNWIND lg.members AS mb
FOREACH (_ IN case when mb.type = 'LocalGroup' then [1] else [] end|
	MERGE (localSubGroup:LocalGroup {name:mb.name, sid:mb.SID, computerName:computer.name})
	MERGE (computer)-[:hasGroup]->(localSubGroup)
	MERGE (localGroup)-[:hasMember]->(localSubGroup)
)
FOREACH (_ IN case when mb.type = 'LocalUser' then [1] else [] end|
	MERGE (localUser:LocalUser {name:mb.name, sid:mb.SID, computerName:computer.name})
	MERGE (computer)-[:hasUser]->(localUser)
	MERGE (localGroup)-[:hasMember]->(localUser)
)
FOREACH (_ IN case when mb.type = 'DomainUser' then [1] else [] end|
	MERGE (domainUser:DomainUser {name:mb.name, sid:mb.SID})
	MERGE (localGroup)-[:hasMember]->(domainUser)
)
FOREACH (_ IN case when mb.type = 'DomainGroup' then [1] else [] end|
	MERGE (domainGroup:DomainGroup {name:mb.name, sid:mb.SID})
	MERGE (localGroup)-[:hasMember]->(domainGroup)
);
''' % pathlib.Path().absolute().as_uri()
out = conn.query(query)

Get all data

In [18]:
query = '''MATCH (n) RETURN n'''
out = conn.query(query)
out

[<Record n=<Node id=0 labels=frozenset({'Computer'}) properties={'PrimaryGroupSid': 'S-1-5-21-3712873460-3785406585-1658148875-515', 'haslaps': False, 'operatingsystem': 'Windows Server 2019 Datacenter Evaluation', 'unconstraineddelegation': False, 'enabled': True, 'sid': 'S-1-5-21-3712873460-3785406585-1658148875-1111', 'tier': 1, 'domain': 'HOT.LOCAL', 'pwdlastset': 1592308245, 'name': 'HOT-JUMPHOST.HOT.LOCAL', 'distinguishedname': 'CN=HOT-JUMPHOST,OU=Jumphost,DC=hot,DC=local', 'serviceprincipalnames': ['WSMAN/hot-jumphost', 'WSMAN/hot-jumphost.hot.local', 'TERMSRV/HOT-JUMPHOST', 'TERMSRV/hot-jumphost.hot.local', 'RestrictedKrbHost/HOT-JUMPHOST', 'HOST/HOT-JUMPHOST', 'RestrictedKrbHost/hot-jumphost.hot.local', 'HOST/hot-jumphost.hot.local'], 'lastlogontimestamp': 1592307353}>>,
 <Record n=<Node id=1 labels=frozenset({'Computer'}) properties={'PrimaryGroupSid': 'S-1-5-21-3712873460-3785406585-1658148875-515', 'haslaps': False, 'operatingsystem': 'Windows Server 2019 Datacenter Evaluat

In [32]:
query = '''
MATCH (adminGrp:LocalGroup)
WHERE adminGrp.sid = 'S-1-5-32-544'
WITH adminGrp
MATCH (adminGrp)-[:hasMember*1..]->(x)
WHERE x:DomainUser OR x:DomainGroup
RETURN DISTINCT x.name AS SamAccountName, adminGrp.computerName AS computer
'''
out = conn.query(query)
out

[<Record SamAccountName='Domain Users' computer='HOT-JUMPHOST.HOT.LOCAL'>]